# Hi-Lo Card Game

In [ ]:
import random

class HiLoGame:
    def __init__(self, num_decks=1):
        self.num_decks = num_decks
        self.deck = self.create_deck()
        self.current_card = None

    def create_deck(self):
        suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
        ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'Jack', 'Queen', 'King', 'Ace']
        deck = [(rank, suit) for suit in suits for rank in ranks] * self.num_decks
        random.shuffle(deck)
        return deck

    def draw_card(self):
        if len(self.deck) > 0:
            self.current_card = self.deck.pop()
            return self.current_card
        else:
            return None

    def compare_cards(self, card1, card2):
        ranks = ['Ace', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'Jack', 'Queen', 'King']
        return ranks.index(card1[0]) < ranks.index(card2[0])

    def play(self, agent):
        score = 0
        self.current_card = self.draw_card()

        while self.current_card is not None:
            print(f"Current card: {self.current_card}")
            guess = agent.make_guess(self.current_card)
            next_card = self.draw_card()

            if next_card is None:
                break

            print(f"Next card: {next_card}")

            if (guess == 'h' and self.compare_cards(self.current_card, next_card)) or \
               (guess == 'l' and not self.compare_cards(self.current_card, next_card)):
                score += 1
                print("Correct guess!")
            else:
                print("Incorrect guess.")
                break

            self.current_card = next_card

        print(f"Game over. Final score: {score}")
        return score

## Agents

In [ ]:
class RandomAgent:
    def make_guess(self, current_card):
        return random.choice(['h', 'l'])

## Playing the game

In [ ]:
if __name__ == "__main__":
    game = HiLoGame()
    agent = RandomAgent()
    game.play(agent)